In [171]:
import numpy as np
import pandas as pd
from utils import * # Functions written by team ghostbusters
import seaborn as sns
from category_encoders import TargetEncoder
import networkx as nx

pd.options.display.float_format = '{:,.2f}'.format

In [172]:
import warnings
warnings.filterwarnings('ignore')

In [173]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [174]:
df = pd.read_csv('./data/test.csv')

<h2>General Attributes</h2>

<h3>Property type</h3>
Ordinal encoding property_type based on EDA result

In [175]:
"""
Generalize property type
"""
df['general_property_type'] = df['property_type'].apply(generalize_property_type) # 3 general types： landed hdb condo

"""
Standardize property type
"""
df['property_type'] = df['property_type'].apply(standardize_property_type) # lower case

In [176]:
"""
Property categori ordinal encoding
The order of encoding follows the EDA results of price_sqft from small to large
"""
ordered_ppt_type = {
"hdb":1, 
"executive condo":2, "walk-up":2,"shophouse":2,
"condo":3,"apartment":3,
"townhouse":4,"terraced house":4,"landed":4,"cluster house":4,
"corner terrace":5,
"conservation house":6,
"semi-detached house":7,
"bungalow":8,
"land only":9,
 }
df["property_type_ordinal"] = df["property_type"].replace(ordered_ppt_type)

<h3>Size_sqft</h3>
Fill outlier with mean value based on general property types (hdb, condo, landed)

In [177]:
"""
Clean size_sqft
"""
#   Find large outlier, transfrom small outlier to size square feet if can find corresponding data in the same property
df.size_sqft = df.apply(lambda r: standardize_size(r, df, 400, 6000,'hdb'), axis=1)
df.size_sqft = df.apply(lambda r: standardize_size(r, df, 400, 1000000,'condo'), axis=1)
df.size_sqft = df.apply(lambda r: standardize_size(r, df, 400, 60000,'landed'), axis=1)
print("Nan and outlier", df.size_sqft.isna().sum())

#   If fill na with mean size_sqft of general property_type (hdb, condo, landed)
df["size_sqft"] = df.groupby(["general_property_type"])["size_sqft"].transform(lambda x: x.fillna(x.mean()))
print("Nan and outlier after filled", df.size_sqft.isna().sum())
print('Num after clean', df.shape[0])

Nan and outlier 28
Nan and outlier after filled 0
Num after clean 6966


<h3>Num_beds</h3>
Fill missing value following EDA observations

In [178]:
"""
Bed
"""
print('Null in num_beds:',len(df[df.num_beds.isnull()]))

#   1. Set studio bed
is_studio = df.title.str.contains('studio')# is studio
is_null = df.num_beds.isnull() # num_beds is null
is_small = df.size_sqft <=900 # is studio but not studio house
df.loc[is_studio&is_null&is_small,"num_beds"] = 1

#   2. Fill with information of the same property
df['num_beds'] = df.groupby(['property_name','size_sqft'])['num_beds'].transform(lambda x: x.fillna(next(iter(x.mode()), np.nan)))

#   3. Fill num_beds with hdb info, in comparison with other type of house, the floor plan and size of hdb is generally fiexed
is_null = df.num_beds.isnull() # num_beds is null
is_4rm = df.size_sqft > 1290 # Observation from EDA over 85% of 4 room larger than 1290 sqft
is_2rm = df.size_sqft < 900 # Observation from EDA over 95% of 2 room smaller than 900 sqft
is_hdb = df.general_property_type.str.contains("hdb", na=False, case=False) # is hdb
df.loc[is_hdb&is_null&is_4rm,'num_beds'] = 4
df.loc[is_hdb&is_null&(~is_2rm)&(~is_4rm),'num_beds'] = 3
df.loc[is_hdb&is_null&is_2rm,'num_beds'] = 2

#   4. Fill the rest with mean based on size_sqft
df["num_beds"] = df.groupby(["size_sqft"])["num_beds"].transform(lambda x: x.fillna(int(x.mean())))
print('data num after cleaning:',df.shape[0])
print('Null in num_beds after cleaned:',len(df[df.num_beds.isnull()]))

Null in num_beds: 35
data num after cleaning: 6966
Null in num_beds after cleaned: 0


<h3>Num_baths</h3>
Fill missing values, fill outlier and other missing data with mean

In [179]:
"""
baths
"""
print('data num before cleaning:',df.shape[0])
print('Null in num_baths:',len(df[df.num_baths.isnull()]))

#   1. Fill with information of the same property
df['num_baths'] = df.groupby(['property_name','num_beds','size_sqft'])['num_baths'].transform(lambda x: x.fillna(next(iter(x.mode()), np.nan)))

#  2. Fill with mode with corresponding num_beds
df['num_baths'] = df.groupby(['num_beds','size_sqft'])["num_baths"].transform(lambda x: x.fillna(next(iter(x.mode()), np.nan)))# Strict search with size_sqft
df['num_baths'] = df.groupby(['num_beds'])["num_baths"].transform(lambda x: x.fillna(next(iter(x.mode()), np.nan)))# Relax search only consider floor plan
print('data num after cleaning:',df.shape[0])
print('Null in num_baths after clean:',len(df[df.num_baths.isnull()]))

data num before cleaning: 6966
Null in num_baths: 149
data num after cleaning: 6966
Null in num_baths after clean: 0


<h3> Remove outliers regarding num_beds and num_baths</h3>
Find outlier based on bed2bath ratio, fill outlier with reference values from same property or mean based on size and floor plans

In [180]:
##  1. Try filling noisy data with information of the same property, outliers are defined by based on boxplot result
df_noise = df.copy()
df_noise['bed2bath'] =df_noise['num_beds']/df_noise['num_baths']
df_noise_cleaned = df_noise.apply(lambda r: standardize_bednbath(r, df_noise, 0.4,3), axis=1)

df_noise_arr = np.array([*df_noise_cleaned])
df['num_beds'] = df_noise_arr[:,0]
df['num_baths'] = df_noise_arr[:,1]
# 2. Fill NaN with mean based on size and floor plan
df["num_beds"] = df.groupby(["size_sqft"])["num_beds"].transform(lambda x: x.fillna(int(x.mean())))
df['num_baths'] = df.groupby(['num_beds'])["num_baths"].transform(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
print('Null in num_beds after clean:',df.num_beds.isna().sum())
print('Null in num_baths after clean:',df.num_baths.isna().sum())

Null in num_beds after clean: 0
Null in num_baths after clean: 0


<h3>Built Year</h3>
Fill missing value

In [181]:
"""
Built year 
"""
##  Fill missing value based on mean values of each general property type 'hdb' 'condo' 'landed'
year_group = df.groupby(by = ['general_property_type'])['built_year'].transform(lambda x: int(x.mean()))
df['built_year'] = df['built_year'].fillna(year_group)

<h3> Tenure </h3>
Group tenure years, ordinal encoding based on EDA results

In [182]:
"""
Tenure
"""
# Genralize tenur into 3 groups 'Nan', '99~110 year' ,'900+ year', 'freehold'
# Ordinal encoding following EDA results
df['tenure_group'] = df['tenure'].apply(standardize_tenure)

<h3>Target encoding per price based on subzone</h3>
Using target encoding information from cleaned train data

In [183]:
"""
load reference train data
"""
df_train_ref = pd.read_csv('./data/train_final_complete_nodrop.csv')

In [184]:
"""
Subzone name encode
"""
# Find corresponding entries in train data to fill test data
df['subzone_per_price_encoded'] = df.apply(lambda r: set_encoded_with_csv(r, df_train_ref, 'subzone_per_price_encoded',"subzone"),axis =1)


<h2>Auxiliary Data</h2>

<h3>MRT</h3>
find 50 mrt stations with the highest in/out degree centrality

In [185]:
# Load data
df_mrt_connections = pd.read_csv('data/auxiliary-data/sg-mrt-connections.csv')
df_mrt = pd.read_csv('data/auxiliary-data/sg-mrt-stations.csv')

In [187]:
# Find important statinons 
G_undirected = nx.Graph()

for idx, row in df_mrt_connections.iterrows():
    G_undirected.add_edge(row['to'], row['from'])

nx_degree_scores = nx.algorithms.centrality.degree_centrality(G_undirected)

ordered_degree_scores = sorted(nx_degree_scores.items(), key=lambda kv: kv[1], reverse=True)

for station, score in ordered_degree_scores[:5]:
    print('{} ({:.5f})'.format(station, score))

important_mrt_stations = [entry[0] for entry in ordered_degree_scores[:50]]
print(important_mrt_stations)

df_important_mrt = df_mrt[df_mrt['name'].isin(important_mrt_stations)]

dhoby ghaut (0.03968)
macpherson (0.03175)
little india (0.03175)
buona vista (0.03175)
chinatown (0.03175)
['dhoby ghaut', 'macpherson', 'little india', 'buona vista', 'chinatown', 'botanic gardens', 'newton', 'serangoon', 'bugis', 'bishan', 'outram park', 'woodlands', 'promenade', 'paya lebar', 'tampines', 'raffles place', 'caldecott', 'expo', 'tanah merah', 'jurong east', 'bayfront', 'marina bay', 'city hall', 'tanjong pagar', 'bright hill', 'mayflower', 'mattar', 'rochor', 'one-north', 'kent ridge', 'downtown', 'telok ayer', 'ubi', 'dover', 'farrer park', 'boon keng', 'clarke quay', 'kaki bukit', 'farrer road', 'holland village', 'harbourfront', 'telok blangah', 'marsiling', 'kranji', 'sengkang', 'buangkok', 'stevens', 'bedok', 'yew tee', 'bukit batok']


In [188]:
df = add_distance_to_nearest_mrt(df_important_mrt, df, 'dist_to_nearest_important_mrt')
df['dist_to_nearest_important_mrt_rounded'] = df['dist_to_nearest_important_mrt'].round(0).astype(int)

<h3>Shopping Mall</h3>
Find number of shopping malls within 300 m of the property

In [189]:
#   Load Data
df_shopping_mall = pd.read_csv('./data/auxiliary-data/sg-shopping-malls.csv')

In [190]:
#   Add number of nearby shopping malls
df = add_number_of_nearby_shopping_malls(df_shopping_mall,df)

<h3>Schools</h3>
Find number of nearby schools within 1 km of the property

In [191]:
df_primary_schools = pd.read_csv('./data/auxiliary-data/sg-primary-schools.csv')
df_secondary_schools = pd.read_csv('./data/auxiliary-data/sg-secondary-schools.csv')

In [192]:
df = add_number_of_nearby_primary_schools(df_primary_schools, df)

In [193]:
df = add_number_of_nearby_secondary_schools(df_secondary_schools, df)

In [194]:
df.num_baths.isna().sum()

0

<h3>Commertial Centre</h3>
Find name of nearest commercial centre(type: BN and CR) within 10 km of the property.
Ordinal encoding the name 

In [195]:
# Load data
df_cc = pd.read_csv('./data/auxiliary-data/sg-commerical-centres.csv')

In [196]:
# Find name of nearest 'BN' commertial centre within 10 km of the property
df= add_name_of_nearest_commercial_centre_by_type(df,df_cc,'BN',10)
# Find name of nearest 'CR' commertial centre within 10 km of the property
df = add_name_of_nearest_commercial_centre_by_type(df,df_cc,'CR',10)

In [197]:
#   Categorical encoding BN
ordered_BN = {
'Novena':9,
'Alexandra':8,
 'Buona Vista':7,
 'Paya Lebar Central':6,'Serangoon':5,
 'Bishan':4,
 'Changi East Urban District':3,
 'Tao Payoh':2,
 'None':1
 }
df["name_of_nearest_BN_ordinal"] = df["name_of_nearest_BN"].replace(ordered_BN)

In [198]:
#   Categorical encoding CR
ordered_CR = {
'Central Business District':5,
 'Jurong Lake District':4,
 'Tampines Regional Centre':3,
 'Seletar Regional Centre':2,
 'Woodlands Regional Centre':1
 }
df["name_of_nearest_CR_ordinal"] = df["name_of_nearest_CR"].replace(ordered_CR)

In [199]:
#   Fill na (subzone not appeared in train set) using property near certain CR
df['subzone_per_price_encoded'] = df.groupby(["name_of_nearest_CR"])['subzone_per_price_encoded'].transform(lambda x: x.fillna(next(iter(x.mode()), np.nan)))

<h2> Drop Columns</h2>
Drop features that are not useful for training

In [201]:
df_final = df[['built_year', 'num_beds', 'num_baths', 'lat', 'lng', 'size_sqft',
                    'tenure_group', 'subzone_per_price_encoded',
                    'property_type_ordinal',
                    #mrt
                    'dist_to_nearest_important_mrt_rounded',
                    #schools
                    'number_of_nearby_primary_schools', 
                    'number_of_nearby_secondary_schools', 
                    #shopping mall
                    'number_of_nearby_shopping_malls',
                    #CR
                    'name_of_nearest_BN_ordinal',
                    'name_of_nearest_CR_ordinal'
                    ]]

In [202]:
# Check is still contains missing values
for col in df_final.columns:
    if df_final[col].isna().sum():
        print(col)

<h2> Save to CSV</h2>

In [203]:
# testset CSV for final prediction
df_final.to_csv('./data/test_final_complete_cleaned.csv',index=False)